In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role

In [2]:
region = boto3.Session().region_name
sm_rt = boto3.Session().client('runtime.sagemaker', region_name=region)

ep_name = 'eeg-clean-model'
id_name = ''
target = 'Eye'

file_name = "eeg-clean-test.csv"
test_file_name = "eeg-clean-test-no-label.csv"

kaggle_file_name = ""
kaggle_test_file_name = ""
kaggle_pred_file_name = ""

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv(file_name)
df = df.drop([target],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(test_file_name, index=False)

       AF3       F7       F3      FC5       T7       P7       O1       O2  \
0  4343.59  3971.79  4265.13  4092.31  4326.67  4618.97  4100.00  4626.15   
1  4431.79  4072.31  4304.62  4154.36  4350.77  4621.03  4077.44  4601.54   
2  4316.41  4037.44  4247.18  4134.87  4327.69  4592.82  4048.21  4598.46   
3  4285.13  3996.41  4260.51  4108.72  4333.33  4617.95  4064.10  4625.13   
4  4288.72  4018.46  4267.69  4131.79  4338.97  4629.23  4086.15  4616.41   

        P8       T8      FC6       F4       F8      AF4  
0  4205.64  4227.69  4238.46  4309.74  4654.36  4403.08  
1  4211.28  4275.90  4249.74  4321.54  4685.64  4476.41  
2  4190.26  4216.92  4177.44  4257.95  4588.21  4365.64  
3  4203.08  4229.23  4201.54  4281.03  4606.15  4354.36  
4  4191.79  4221.03  4196.41  4270.77  4590.26  4336.92  
(4494, 14)


In [5]:
f = open(test_file_name)
count = 0
pred_label = []
pred_neg_prpb = []
pred_pos_prpb = []

for line in f:
    if count==0:
        count=1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    #print(response)
    pred_label.append(int(response[0]))
    ind1 = response.index("[")
    ind2 = response.index(",", ind1)
    neg_prob = float(response[ind1+1:ind2])
    #print(neg_prob)
    pred_neg_prpb.append(neg_prob)
    ind1 = response.index("]", ind2)
    pos_prob = float(response[ind2+2:ind1])
    #print(pos_prob)
    pred_pos_prpb.append(pos_prob)
    count = count + 1


In [6]:
print(pred_label[:20])
print(pred_neg_prpb[:20])
print(pred_pos_prpb[:20])


import pandas as pd
import numpy as np
pd = pd.read_csv(file_name)
y_true = np.array(pd[target])
y_pred = np.array(pred_label)
print("test accuracy = {}".format(np.mean(y_true==y_pred)))

from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_true, pred_pos_prpb, pos_label=1)
print("test auc = {}".format(metrics.auc(fpr, tpr)))

[1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1]
[0.001852965448051691, 0.9767436981201172, 0.00048117368714883924, 0.9946314096450806, 0.5153131484985352, 0.9758253693580627, 0.9961816668510437, 0.9957191944122314, 0.015399006195366383, 0.0012904751347377896, 0.004676493350416422, 0.9928961992263794, 0.9841356873512268, 0.9997853636741638, 0.9999418258666992, 0.9025738835334778, 0.903184175491333, 1.860418160504196e-05, 0.9986171722412109, 1.845318911364302e-05]
[0.9981470704078674, 0.02325613796710968, 0.9995189905166626, 0.005368595011532307, 0.48468682169914246, 0.024174626916646957, 0.0038183790165930986, 0.004280886612832546, 0.9846011996269226, 0.9987095594406128, 0.9953234791755676, 0.007103772833943367, 0.01586422696709633, 0.00021464283054228872, 5.8086665376322344e-05, 0.0974261611700058, 0.09681590646505356, 0.9999814033508301, 0.0013829792151227593, 0.9999814033508301]
test accuracy = 0.9704049844236761
test auc = 0.9965072364687194


In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv(kaggle_file_name)

kaggle_test_ids = np.array(df[id_name])

df = df.drop([id_name],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(kaggle_test_file_name, index=False)

FileNotFoundError: [Errno 2] File  does not exist: ''

In [14]:
f = open(kaggle_test_file_name)
count = 0
kaggle_pred = []

for line in f:
    if count == 0:
        count = 1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    test_id = kaggle_test_ids[count-1]
    pred = int(response[0])
    kaggle_pred.append([test_id, pred])
    count = count + 1

kaggle_pred_df = pd.DataFrame(kaggle_pred, columns = [id_name, target])

kaggle_pred_df.to_csv(kaggle_pred_file_name, index=False)

In [5]:
print(kaggle_pred_df.head())
print(kaggle_pred_df.shape)

NameError: name 'kaggle_pred_df' is not defined

In [22]:
import boto3
sm = boto3.client('sagemaker')
#sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_endpoint(EndpointName=ep_name)

{'ResponseMetadata': {'RequestId': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 24 Mar 2021 07:51:25 GMT'},
  'RetryAttempts': 0}}